In [1]:
import pandas as pd
import plotly.express as px
import yfinance as yf
from tqdm import tqdm
from datetime import datetime
import plotly.graph_objects as go
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from funcoes import *

# Carregamento

In [3]:
df = pd.read_parquet('base/base_desenvolvimento.parquet.gzip')

In [4]:
df.shape

(140841, 121)

# Analisando Sinal

In [5]:
# isolando acao
df['acao'].unique()

['ABCB4', 'ALUP11', 'ARZZ3', 'B3SA3', 'BBAS3', ..., 'UNIP6', 'USIM5', 'VALE3', 'VIVT3', 'WEGE3']
Length: 65
Categories (65, object): ['ABCB4', 'ALUP11', 'ARZZ3', 'B3SA3', ..., 'USIM5', 'VALE3', 'VIVT3', 'WEGE3']

In [6]:
# amostras
df_pet = df[df['acao'] == 'PETR4']
df_val = df[df['acao'] == 'VALE3']

In [7]:
df_pet.shape, df_val.shape

((2179, 121), (2179, 121))

In [8]:
df_pet.columns

Index(['index', 'Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'acao',
       'CDL_DOJI_10_0.1', 'CDL_INSIDE',
       ...
       'PVR', 'PVT', 'signal', 'label_l1', 'label_l2', 'label_l3', 'mes',
       'dia_semana', 'dia_mes', 'dia_ano'],
      dtype='object', length=121)

In [10]:
df_pet['label_compra'] = np.where(
    df_pet['label_l1'] == 1, df_pet['Close'], np.nan)
df_pet['label_venda'] = np.where(
    df_pet['label_l1'] == 2, df_pet['Close'], np.nan)

df_val['label_compra'] = np.where(
    df_val['label_l1'] == 1, df_val['Close'], np.nan)
df_val['label_venda'] = np.where(
    df_val['label_l1'] == 2, df_val['Close'], np.nan)

C:\Users\sampr\AppData\Local\Temp\ipykernel_1152\1052702407.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pet['label_compra'] = np.where(
C:\Users\sampr\AppData\Local\Temp\ipykernel_1152\1052702407.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pet['label_venda'] = np.where(
C:\Users\sampr\AppData\Local\Temp\ipykernel_1152\1052702407.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See 

## Série histórica

In [11]:
# petr4
fig_candlestick = plot_candlestick(df_pet, x='Date')
# exemplo plot plotly
fig_candlestick.add_trace(
    go.Scatter(
        name='Ponto de Compra',
        x=df_pet['Date'],
        y=df_pet['label_compra'],
        mode='markers',
        marker=dict(color="green"),
        # line=dict(width=1, dash='dash')
    )
)

fig_candlestick.add_trace(
    go.Scatter(
        name='Ponto de Venda',
        x=df_pet['Date'],
        y=df_pet['label_venda'],
        mode='markers',
        marker=dict(color="cyan"),
        # line=dict(width=1, dash='dash')
    )
)
fig_candlestick

In [12]:
# petr4
fig_candlestick = plot_candlestick(df_val, x='Date')
# exemplo plot plotly
fig_candlestick.add_trace(
    go.Scatter(
        name='Ponto de Compra',
        x=df_val['Date'],
        y=df_val['label_compra'],
        mode='markers',
        marker=dict(color="green"),
        # line=dict(width=1, dash='dash')
    )
)

fig_candlestick.add_trace(
    go.Scatter(
        name='Ponto de Venda',
        x=df_val['Date'],
        y=df_val['label_venda'],
        mode='markers',
        marker=dict(color="cyan"),
        # line=dict(width=1, dash='dash')
    )
)
fig_candlestick

# Backtesting 

Teste do sinal pelo backtesting

- Considera um investimento de 1000 reais de 2023-01-01 à 2024-06-30

In [19]:
df_test = df[df['Date'] >= '2023-01-01']

In [13]:
def test_modelos(acao, df_val=None):
    signais = [i for i in df_val.columns if 'signal' in i]
    df_aux = df_val.loc[
        df_val['acao'] == acao,
        ['Date', 'Close']+signais
    ]

    performances = []

    for sinal in signais:
        bkt_l1 = Backtesting(
            df=df_aux,
            sinal=sinal,
            capital_inicial=1000
        )
        bkt_l1.backtesting()
        performance = round(bkt_l1.capital, 1)
        performances.append(performance)
    return performances, signais

In [14]:
import warnings
warnings.filterwarnings('ignore')

In [20]:
acoes = df_test['acao'].unique()
backtest_result = {}
for acao in acoes:
    performance, signais = test_modelos(acao, df_test)
    backtest_result[acao] = performance

In [21]:
df_result = pd.DataFrame(backtest_result, index=[
                         'sinal']).T

In [22]:
df_result.head()

,sinal
ABCB4,4097.2
ALUP11,2686.5
ARZZ3,7414.1
B3SA3,4421.5
BBAS3,3120.3


In [23]:
df_result.sort_values('sinal', ascending=False)

,sinal
BPAN4,13601.7
ENAT3,13292.2
MRFG3,11340.7
POMO3,9538.6
POMO4,9220.5
...,...
UNIP6,2752.8
HYPE3,2696.1
ALUP11,2686.5
TRPL4,2543.1


In [24]:
df_result.describe()

,sinal
count,65.000000
mean,4796.881538
std,2460.194232
min,2156.000000
25%,3154.700000
50%,3903.600000
75%,5642.200000
max,13601.700000


In [25]:
(df_result > 1000).sum() / df_result.shape[0]

sinal    1.0
dtype: float64

todas as ações tiveram retorno positivo 